In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense, BatchNormalization
from keras import backend as K
from keras.optimizers import RMSprop
from keras import regularizers as reg

Using TensorFlow backend.


In [ ]:
from datetime import datetime
global_start=datetime.now()

#Dimensions of our flicker images is 256 X 256
img_width, img_height = 256, 256

#Declaration of parameters needed for training and validation
train_data_dir = 'data/train'
validation_data_dir = 'data/validation'
nb_train_samples = 3600 #1200 training samples for each class
nb_validation_samples = 1200 #400 training samples for each class
epochs = 40
batch_size = 16

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

#First convolution layer
model = Sequential()
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu',kernel_initializer='he_normal',kernel_regularizer=reg.l1_l2(l1=0.001, l2=0.001),input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))

#Second convolution layer
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu',kernel_initializer='he_normal',kernel_regularizer=reg.l1_l2(l1=0.001, l2=0.001),input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))

#Third convolution layer
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu',kernel_initializer='he_normal',kernel_regularizer=reg.l1_l2(l1=0.001, l2=0.001),input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))

#Flatten the outputs of the convolution layer into a 1D contigious array
model.add(Flatten())

#Add a fully connected layer containing 128 neurons, use activation as relu and a dropout rate of 0.5
model.add(Dense(256, activation='relu',kernel_initializer='he_normal',kernel_regularizer=reg.l1_l2(l1=0.001, l2=0.001)))
model.add(BatchNormalization())
model.add(Dropout(rate=0.4))

#Add another fully connected layer containing 128 neurons, use activation as relu and a dropout rate of 0.5
model.add(Dense(256, activation='relu',kernel_initializer='he_normal',kernel_regularizer=reg.l1_l2(l1=0.001, l2=0.001)))
model.add(BatchNormalization())
model.add(Dropout(rate=0.4))

#Add the ouput layer containing 3 neurons, because we have 3 categories
model.add(Dense(3, activation='softmax',kernel_initializer='glorot_uniform'))

optim=RMSprop(lr=0.0001, epsilon=1e-8, decay=1e-6)
model.compile(loss='categorical_crossentropy',optimizer=optim,metrics=['accuracy'])
model.summary()

#We will use the below code snippet for data augmentation on the training data
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.4,
                                   zoom_range=0.4,
                                   vertical_flip=True,
                                   rotation_range=30,
                                   horizontal_flip=True)

#We won't augment the test data. We will just use ImageDataGenerator to rescale.
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_data_dir,
                                                    target_size=(img_width, img_height),
                                                    batch_size=batch_size,
                                                    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(validation_data_dir,
                                                        target_size=(img_width, img_height),
                                                        batch_size=batch_size,
                                                        class_mode='categorical')

model.fit_generator(train_generator,
                    steps_per_epoch=nb_train_samples // batch_size,
                    epochs=epochs,
                    validation_data=validation_generator,
                    validation_steps=nb_validation_samples // batch_size) 


model.save_weights('cnn_from_scratch_weights.h5') 
model.save('cnn_from_scratch_model.h5') #Load using: model = load_model('cnn_from_scratch_weights.h5') from keras.models import load_model
print("Time taken to train the baseline model from scratch: ",datetime.now()-global_start)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 254, 254, 64)      1792      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 127, 127, 64)      0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 125, 125, 64)      36928     
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 62, 62, 64)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 60, 60, 64)        36928     
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 30, 30, 64)        0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 57600)             0         
__________

In [ ]:
import matplotlib.pyplot as plt

#This function is used to plot/update the train and test loss after each epoch.
def plt_dynamic_loss(x, vy, ty, ax, colors=['b']):
    plt.figure(figsize=(10,6))
    ax.plot(x, vy, 'b', label="Validation Loss")
    ax.plot(x, ty, 'r', label="Train Loss")
    plt.legend()
    plt.grid()
    fig.canvas.draw()


#Get model history
history=model.history

#Plot train vs test loss
fig,ax = plt.subplots(1,1)
ax.set_xlabel('Epoch') ; ax.set_ylabel('Categorical Crossentropy Loss')

#List of epoch numbers
x = list(range(1,epochs+1))

#Display the loss
val_loss = history.history['val_loss'] #Validation Loss
loss = history.history['loss'] #Training Loss
plt_dynamic_loss(x, val_loss, loss, ax)